In [ ]:
import torch
import torch.nn as nn

#!! หมายเหตุ: ผมได้อธิบาย แสดง pytorch code class ของโครงสร้าง ในรูปแบบ comment หลังบรรทัด หากเข้าใจโจทย์ผิด ขออภัยมา ณ ที่นี้

# สร้างคลาส CustomCNN ที่สืบทอดจาก nn.Module
class CustomCNN(nn.Module):
    def __init__(self, input_size=(3, 128, 128)):  # input_size คือ (channels, height, width)
        super(CustomCNN, self).__init__()

        # ชุดแรกของ Convolutional Layers
        self.conv1 = nn.Conv2d(in_channels=input_size[0], out_channels=16, kernel_size=3, stride=1, padding=1)  # Conv2D: 3x128x128 → 16x128x128
        self.relu1 = nn.ReLU()  # Activation ReLU
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, stride=1, padding=1)  # Conv2D: 16x128x128 → 16x128x128
        self.relu2 = nn.ReLU()  # Activation ReLU
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # MaxPool: 16x128x128 → 16x64x64

        # ชุดที่สองของ Convolutional Layers
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=64, kernel_size=5, stride=1, padding=0)  # Conv2D: 16x64x64 → 64x60x60
        self.relu3 = nn.ReLU()  # Activation ReLU
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, stride=1, padding=0)  # Conv2D: 64x60x60 → 128x56x56
        self.relu4 = nn.ReLU()  # Activation ReLU
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # MaxPool: 128x56x56 → 128x28x28

        self.flatten = nn.Flatten()  # แปลง tensor เป็น vector สำหรับ Fully Connected

        # Fully Connected Layers
        self.fc1 = nn.Linear(self.get_input_size_fc1(input_size), 200)  # Linear: 128x28x28 (100352) -> 200
        self.relu5 = nn.ReLU()  # Activation ReLU
        self.fc2 = nn.Linear(200, 64)  # Linear: 200 -> 64
        self.relu6 = nn.ReLU()  # Activation ReLU
        self.fc3 = nn.Linear(64, 10)  # Linear: 64 -> 10 (จำนวน class ที่ต้องการ)
    
    def get_input_size_fc1(self, input_shape):  # ฟังก์ชันสำหรับคำนวณขนาด input ให้ fc1
        with torch.no_grad():
            x = torch.zeros(1, *input_shape)  # สร้าง input จำลอง

            x = self.conv1(x)
            x = self.relu1(x)
            x = self.conv2(x)
            x = self.relu2(x)
            x = self.pool1(x)

            x = self.conv3(x)
            x = self.relu3(x)
            x = self.conv4(x)
            x = self.relu4(x)
            x = self.pool2(x)

            x = self.flatten(x)
            return x.size(1)  # ส่งขนาด vector สำหรับ Fully Connected Layer

    def forward(self, x):  # ฟังก์ชันที่กำหนดการไหลของข้อมูล
        x = self.conv1(x)  # Conv Layer 1
        x = self.relu1(x)  # ReLU 1
        x = self.conv2(x)  # Conv Layer 2
        x = self.relu2(x)  # ReLU 2
        x = self.pool1(x)  # MaxPool 1

        x = self.conv3(x)  # Conv Layer 3
        x = self.relu3(x)  # ReLU 3
        x = self.conv4(x)  # Conv Layer 4
        x = self.relu4(x)  # ReLU 4
        x = self.pool2(x)  # MaxPool 2

        x = self.flatten(x)  # Flatten เพื่อเข้า Fully Connected

        x = self.fc1(x)  # Fully Connected 1
        x = self.relu5(x)  # ReLU 5
        x = self.fc2(x)  # Fully Connected 2
        x = self.relu6(x)  # ReLU 6
        x = self.fc3(x)  # Fully Connected 3 (Output)

        return x
